In [175]:
#this is the block of code that generates unrealized profits for companies for which portfolia holder has bought 
#and sold the stocks both.]\

import pandas as pd
from fpdf import FPDF
import yfinance as yf

# Load the Excel file
df_xlsx = pd.read_excel("Book2.xlsx")

# Filter buy and sell data
data_for_buy = df_xlsx[df_xlsx['TYPE'] == 'Buy']
data_for_sell = df_xlsx[df_xlsx['TYPE'] == 'Sell']

# Summarize the quantities by ticker
buy_summary = data_for_buy.groupby('EXCHANGE CODE: TICKER')['UNITS'].sum().to_dict()
sell_summary = data_for_sell.groupby('EXCHANGE CODE: TICKER')['UNITS'].sum().to_dict()
buy_price_dict = data_for_buy.groupby('EXCHANGE CODE: TICKER')['AMOUNT'].sum().to_dict()

sell_price_dict = data_for_sell.groupby('EXCHANGE CODE: TICKER')['AMOUNT'].sum().to_dict()

# Function to fetch the current stock price using yfinance for Indian stocks
def get_stock_price(ticker):
    try:
        # Assuming NSE symbols, append '.NS' to the ticker. Adjust if using BSE.
        stock = yf.Ticker(f"{ticker[4:]}.NS")  # Use '.BO' for BSE
        price = stock.history(period="1d")['Close'].iloc[-1]  # Get the latest closing price
        return round(float(price), 2)
    except Exception as e:
        print(f"Error fetching price for {ticker[4:]}: {e}")
        return None

net_quant = {key: round((buy_summary[key] - sell_summary[key]), 5) for key in sell_summary if round((buy_summary[key] - sell_summary[key]), 5) != 0}
difference_dict = {key: ((buy_price_dict[key]-sell_price_dict[key])/net_quant[key]) for key in net_quant}

# Calculate the remaining stock and get the current price
result = {}
for key in difference_dict:
    buy_quantity = buy_summary.get(key, 0)
    sell_quantity = sell_summary[key]
    remaining_quantity = buy_quantity - sell_quantity
    
    # Only process if remaining quantity is greater than zero
    if remaining_quantity != 0:
        buy_price = difference_dict.get(key, 0)
        current_price = get_stock_price(key)
        
        if current_price is not None:
            current_value = round(remaining_quantity * current_price, 2)
            unrealized_profit = current_value-(buy_price*net_quant[key])
            total_cost_of_remaining_shares = buy_price * remaining_quantity
            profit_percentage = (unrealized_profit / (buy_price*net_quant[key])) * 100 
            result[key] = [
                round(remaining_quantity, 10),  # Remaining quantity
                current_price,  # Current price of the stock
                round(buy_price,5), #price at which the stock was bought
                round(unrealized_profit, 2),  # Unrealized profit
                current_value,  # Current value of the portfolio/investment
                round(profit_percentage, 2)  # Profit percentage
            ]

# Print the result
print(result)


{'NSE:ABBOTINDIA': [0.68, 27309.85, 19403.41176, 5376.38, 18570.7, 40.75], 'NSE:BAJEL': [46.35, 271.3, 206.11823, 3021.17, 12574.75, 31.62], 'NSE:BBTC': [-0.005, 2242.55, 2425791.0, 12117.75, -11.21, -99.91], 'NSE:BIKAJI': [30.16, 849.7, 411.90821, 13203.8, 25626.95, 106.28], 'NSE:HCLTECH': [14.04, 1592.65, 1520.05698, 1019.21, 22360.81, 4.78], 'NSE:MAYURUNIQ': [24.32, 647.9, 508.91077, 3380.22, 15756.93, 27.31], 'NSE:METROPOLIS': [7.13, 2010.55, 988.71304, 7285.7, 14335.22, 103.35], 'NSE:MSUMI': [257.97, 69.9, 55.73838, 3653.27, 18032.1, 25.41], 'NSE:PRAJIND': [20.76, 736.4, 357.31792, 7869.74, 15287.66, 106.09], 'NSE:SRF': [6.22, 2521.05, 2735.22508, -1332.17, 15680.93, -7.83], 'NSE:WABAG': [8.31, 1300.6, 599.65945, 5824.82, 10807.99, 116.89], 'NSE:WIPRO': [29.22, 490.5, 344.42813, 4268.22, 14332.41, 42.41], 'NSE:YESBANK': [608.24, 24.11, 18.93675, 3146.58, 14664.67, 27.32]}


In [176]:
from fpdf import FPDF

pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=10)
pdf.add_page()

# Add a background color for the title
pdf.set_fill_color(200, 220, 255)
pdf.set_font("Arial", "B", 12)  # Smaller font size for the title
pdf.cell(0, 12, txt="Stock Portfolio Summary", ln=True, align="C", fill=True)
pdf.ln(10)
pdf.set_right_margin(10)

# Set column headers with a background color and border
pdf.set_fill_color(105, 105, 105)  # Dark gray
pdf.set_text_color(255, 255, 255)  # White text for headers
pdf.set_font("Arial", "B", 10)  # Smaller font size for headers

# Adjust the width of each column so that it fits within the page
pdf.cell(25, 10, txt="Stock", border=1, align="C", fill=True)
pdf.cell(30, 10, txt="Net Quantity", border=1, align="C", fill=True)
pdf.cell(25, 10, txt="Buy Price", border=1, align="C", fill=True)
pdf.cell(25, 10, txt="Current Price", border=1, align="C", fill=True)
pdf.cell(25, 10, txt="Profit/Loss", border=1, align="C", fill=True)
pdf.cell(30, 10, txt="Profit/Loss%", border=1, align="C", fill=True)
pdf.cell(35, 10, txt="Current Value", border=1, align="C", fill=True)
pdf.ln()

# Add data rows
pdf.set_font("Arial", "", 10)  # Smaller font size for data rows
pdf.set_text_color(0, 0, 0)  # Black text for data

for idx, (key, value) in enumerate(result.items()):
    # Alternate row background color for better readability
    if idx % 2 == 0:
        pdf.set_fill_color(245, 245, 245)  # Light gray
        fill = True
    else:
        fill = False

    # Add stock name and quantities
    pdf.cell(25, 10, txt=key[4:], border=1, align="C", fill=fill)  # Stock name without the exchange prefix
    pdf.cell(30, 10, txt=str(value[0]), border=1, align="C", fill=fill)  # Total quantities
    
    # Add buy price
    pdf.cell(25, 10, txt=str(value[2]), border=1, align="C", fill=fill)  # Buy price
    
    # Add current price
    pdf.cell(25, 10, txt=str(value[1]), border=1, align="C", fill=fill)  # Current price

    # Add profit with color coding
    if value[3] >= 0:
        pdf.set_text_color(0, 128, 0)  # Green for profit
    else:
        pdf.set_text_color(255, 0, 0)  # Red for loss
    pdf.cell(25, 10, txt=str(value[3]), border=1, align="C", fill=fill)  # Profit/Loss
    pdf.set_text_color(0, 0, 0)  # Reset text color to black
    
    # Add profit percentage and current value
    pdf.cell(30, 10, txt=str(value[5]) + '%', border=1, align="C", fill=fill)  # Profit percentage
    pdf.cell(35, 10, txt=str(value[4]), border=1, align="C", fill=fill)  # Current value
    pdf.ln()

# Add a footer
pdf.set_y(-15)
pdf.set_font("Arial", "I", 8)  # Footer font size
pdf.set_text_color(128, 128, 128)
pdf.cell(0, 10, txt="End of Report", align="C")

# Save the PDF to a file
pdf.output("stock_portfolio3.pdf")

print("PDF created successfully.")


PDF created successfully.


In [226]:
#this is the block of code that generates unrealized profits for companies tfor which portfolia holder has only bought stocks as
#well as bought and sold the stocks both.

import pandas as pd
from fpdf import FPDF
import yfinance as yf

# Load the Excel file
df_xlsx = pd.read_excel("Book2.xlsx")

# Filter buy and sell data
data_for_buy = df_xlsx[df_xlsx['TYPE'] == 'Buy']
data_for_sell = df_xlsx[df_xlsx['TYPE'] == 'Sell']

# Summarize the quantities by ticker
buy_summary = data_for_buy.groupby('EXCHANGE CODE: TICKER')['UNITS'].sum().to_dict()
sell_summary = data_for_sell.groupby('EXCHANGE CODE: TICKER')['UNITS'].sum().to_dict()
buy_price_dict = data_for_buy.groupby('EXCHANGE CODE: TICKER')['AMOUNT'].sum().to_dict()

sell_price_dict = data_for_sell.groupby('EXCHANGE CODE: TICKER')['AMOUNT'].sum().to_dict()



'''generating the introductory data for the client ie the current market value and the total investment value and other things'''
total_investment_value_data = (df_xlsx[df_xlsx['TYPE']=='Cash Deposit'])
valc = list((total_investment_value_data.groupby('EXCHANGE CODE: TICKER')['PRICE'].sum().to_dict()).values())
total_investment_value = valc[0] #gets the total amount of money that is invested by the portfolio holder

value_buy_sum = data_for_buy['Value'].sum()
value_sell_sum = data_for_sell['Value'].sum()
total_value = round(value_buy_sum + value_sell_sum,4)
#print(total_value)

cash_balance_buy_sum = data_for_buy['Cash Balance'].sum()
cash_balance_sell_sum = data_for_sell['Cash Balance'].sum()
total_cash_balance = round(cash_balance_buy_sum + cash_balance_sell_sum,4)
#print(total_cash_balance)
market_value = round(total_cash_balance + total_value,4) # gets the total market value 


'''generates a table of all the stock holdings'''
# Function to fetch the current stock price using yfinance for Indian stocks
def get_stock_price(ticker):
    try:
        # Assuming NSE symbols, append '.NS' to the ticker. Adjust if using BSE.
        stock = yf.Ticker(f"{ticker[4:]}.NS") if ticker[0:3] == 'NSE' else yf.Ticker(f"{ticker[4:]}.BO") 
        price = stock.history(period="1d")['Close'].iloc[-1]  # Get the latest closing price
        return round(float(price), 2)
    except Exception as e:
        print(f"Error fetching price for {ticker[4:]}: {e}")
        return None

net_quant = {key: round((buy_summary[key] - sell_summary.get(key, 0)), 5) for key in buy_summary if round((buy_summary[key] - sell_summary.get(key, 0)), 5) != 0}
difference_dict = {key: ((buy_price_dict[key]-sell_price_dict.get(key, 0))/net_quant[key]) for key in net_quant}

# Calculate the remaining stock and get the current price
result = {}
for key in difference_dict:
    buy_quantity = buy_summary.get(key, 0)
    sell_quantity = sell_summary.get(key, 0)
    remaining_quantity = buy_quantity - sell_quantity
    
    # Only process if remaining quantity is greater than zero
    if remaining_quantity != 0:
        buy_price = difference_dict.get(key, 0)
        current_price = get_stock_price(key)
        
        if current_price is not None:
            current_value = round(remaining_quantity * current_price, 2)
            unrealized_profit = current_value - (buy_price * remaining_quantity)
            total_cost_of_remaining_shares = buy_price * remaining_quantity
            profit_percentage = (unrealized_profit / total_cost_of_remaining_shares) * 100 
            result[key] = [
                round(remaining_quantity, 10),  # Remaining quantity
                current_price,  # Current price of the stock
                round(buy_price,5),  # Price at which the stock was bought
                round(unrealized_profit, 2),  # Unrealized profit
                current_value,  # Current value of the portfolio/investment
                round(profit_percentage, 2)  # Profit percentage
            ]

# Print the result
print(result)




{'NSE:ABBOTINDIA': [0.68, 27053.95, 19403.41176, 5202.37, 18396.69, 39.43], 'NSE:APOLLOHOSP': [3.6, 6515.85, 5972.0, 1957.86, 23457.06, 9.11], 'NSE:ASHOKLEY': [66.85, 246.45, 224.23897, 1484.81, 16475.18, 9.91], 'NSE:BAJEL': [46.35, 266.15, 206.11823, 2782.47, 12336.05, 29.12], 'NSE:BBTC': [-0.005, 2272.3, 2425791.0, 12117.59, -11.36, -99.91], 'NSE:BIKAJI': [30.16, 835.35, 411.90821, 12771.01, 25194.16, 102.8], 'NSE:CENTURYPLY': [21.42, 730.6, 691.9, 828.95, 15649.45, 5.59], 'NSE:HAVELLS': [15.42, 1845.95, 1620.0, 3484.15, 28464.55, 13.95], 'NSE:HCLTECH': [14.04, 1626.15, 1520.05698, 1489.55, 22831.15, 6.98], 'NSE:IEX': [92.52, 185.81, 148.0, 3498.18, 17191.14, 25.55], 'NSE:KSCL': [15.4, 1047.0, 832.17403, 3308.32, 16123.8, 25.82], 'NSE:MARUTI': [1.78, 12205.65, 12793.14607, -1045.74, 21726.06, -4.59], 'NSE:MAYURUNIQ': [24.32, 643.8, 508.91077, 3280.51, 15657.22, 26.51], 'NSE:METROPOLIS': [7.13, 1981.35, 988.71304, 7077.51, 14127.03, 100.4], 'NSE:MSUMI': [257.97, 69.01, 55.73838, 3423.

In [224]:
from fpdf import FPDF

pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=10)
pdf.add_page()

# Add a background color for the title
pdf.set_fill_color(200, 220, 255)
pdf.set_font("Arial", "B", 12)  # Smaller font size for the title
pdf.cell(0, 12, txt="Stock Portfolio Summary", ln=True, align="C", fill=True)
pdf.ln(10)
pdf.set_right_margin(10)

# Set column headers with a background color and border
pdf.set_fill_color(105, 105, 105)  # Dark gray
pdf.set_text_color(255, 255, 255)  # White text for headers
pdf.set_font("Arial", "B", 10)  # Smaller font size for headers

# Adjust the width of each column so that it fits within the page
pdf.cell(25, 10, txt="Stock", border=1, align="C", fill=True)
pdf.cell(30, 10, txt="Net Quantity", border=1, align="C", fill=True)
pdf.cell(25, 10, txt="Buy Price", border=1, align="C", fill=True)
pdf.cell(25, 10, txt="Current Price", border=1, align="C", fill=True)
pdf.cell(25, 10, txt="Profit/Loss", border=1, align="C", fill=True)
pdf.cell(30, 10, txt="Profit/Loss%", border=1, align="C", fill=True)
pdf.cell(35, 10, txt="Current Value", border=1, align="C", fill=True)
pdf.ln()

# Add data rows
pdf.set_font("Arial", "", 10)  # Smaller font size for data rows
pdf.set_text_color(0, 0, 0)  # Black text for data

for idx, (key, value) in enumerate(result.items()):
    # Alternate row background color for better readability
    if idx % 2 == 0:
        pdf.set_fill_color(245, 245, 245)  # Light gray
        fill = True
    else:
        fill = False

    # Add stock name and quantities
    pdf.cell(25, 10, txt=key[4:], border=1, align="C", fill=fill)  # Stock name without the exchange prefix
    pdf.cell(30, 10, txt=str(value[0]), border=1, align="C", fill=fill)  # Total quantities
    
    # Add buy price
    pdf.cell(25, 10, txt=str(value[2]), border=1, align="C", fill=fill)  # Buy price
    
    # Add current price
    pdf.cell(25, 10, txt=str(value[1]), border=1, align="C", fill=fill)  # Current price

    # Add profit with color coding
    if value[3] >= 0:
        pdf.set_text_color(0, 128, 0)  # Green for profit
    else:
        pdf.set_text_color(255, 0, 0)  # Red for loss
    pdf.cell(25, 10, txt=str(value[3]), border=1, align="C", fill=fill)  # Profit/Loss
    pdf.set_text_color(0, 0, 0)  # Reset text color to black
    
    # Add profit percentage with color coding
    if value[5] >= 0:
        pdf.set_text_color(0, 128, 0)  # Green for positive percentage
    else:
        pdf.set_text_color(255, 0, 0)  # Red for negative percentage
    pdf.cell(30, 10, txt=str(value[5]) + '%', border=1, align="C", fill=fill)  # Profit percentage
    
    # Reset text color to black and add current value
    pdf.set_text_color(0, 0, 0)
    pdf.cell(35, 10, txt=str(value[4]), border=1, align="C", fill=fill)  # Current value
    pdf.ln()

# Add a footer
pdf.set_y(-15)
pdf.set_font("Arial", "I", 8)  # Footer font size
pdf.set_text_color(128, 128, 128)
pdf.cell(0, 10, txt="End of Report", align="C")

# Save the PDF to a file
pdf.output("stock_portfolio2.pdf")

print("PDF created successfully.")


PDF created successfully.
